In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle

In [ ]:
import json
json_path = "/content/drive/MyDrive/Fundus/EyePACS/kaggle.json"
f = open(json_path, 'r')
kaggle_details = json.load(f)
f.close()
kaggle_user = kaggle_details["username"]
kaggle_key = kaggle_details["key"]

In [ ]:
!export KAGGLE_USERNAME=kaggle_user
!export KAGGLE_KEY=kaggle_key

In [ ]:
%cd /content

/content


In [ ]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/drive/MyDrive/Fundus/EyePACS/kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
%cd /content/drive/MyDrive/Fundus/
!ls

/content/drive/MyDrive/Fundus
DeepDRiD  EyePACS  messidor_densenet_full_best.zip  Models  Pickles


In [ ]:
%cd EyePACS

/content/drive/MyDrive/Fundus/EyePACS


In [ ]:
!kaggle competitions download diabetic-retinopathy-detection -f train.zip.005.zip

404 - Not Found - Not found


In [ ]:
%cd ..

/content/drive


In [ ]:
%cd ..

/content


In [ ]:
import torch
from torchvision import models
from torchvision import transforms

In [ ]:
use_cuda = torch.cuda.is_available()
device = 'cuda' if use_cuda else 'cpu'

In [ ]:
import pandas as pd

def get_file_paths(path, base_path, image_path_postfix):
  df = pd.read_csv(path)
  image_id_list, image_path_list, eye_lvl_list = list(df["image_id"]), list(df["image_path"]), list(df["eye_DR_Level"])
  image_path_list = [s.replace('\\', '/') for s in image_path_list]

  image_path_list_final = []
  for s in image_path_list:
    splits = s.split('/')
    sf = '/' + splits[1] + '/' + image_path_postfix
    postfix = '/'.join(splits[2:])
    sf = sf + '/' + postfix
    image_path_list_final.append(sf)

  file_paths = [(image_id_list[i], base_path + image_path_list_final[i], eye_lvl_list[i]) for i in range(len(image_id_list))]
  return file_paths

In [ ]:
def make_batches(list_obj, batch_size):
  j = 0
  i = batch_size
  if i > len(list_obj):
    return [list_obj]
  toret = []
  for i in range(batch_size, len(list_obj), batch_size):
    toret.append(list_obj[j : i])
    j = i
  return toret

In [ ]:
base_path = "/content/drive/MyDrive/Fundus"

In [ ]:
base_path = "/content/drive/MyDrive/Fundus"
ckpt_path = base_path + '/EyePACS/resnet50_128_08.pt'

weights = torch.load(ckpt_path)
model = models.resnet50()
# Weights of fully connected layer are removed in the file, so set strict to be False.
model.load_state_dict(weights, strict=False)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
model = model.to(device)

In [ ]:
drid_path = base_path + "/DeepDRiD"
train_csv_path = drid_path + "/training_data.csv"
valid_csv_path = drid_path + "/validation_data.csv"
image_path_prefix = "Images"

file_items_train = get_file_paths(train_csv_path, drid_path, image_path_prefix)
file_items_valid = get_file_paths(valid_csv_path, drid_path, image_path_prefix)

batch_size = 10

#train_paths_batched = make_batches(file_items_train, batch_size)
#valid_paths_batched = make_batches(file_items_valid, batch_size)

In [ ]:
fit = get_file_paths(train_csv_path, drid_path, image_path_prefix)
print(fit == file_items_train)

True


In [ ]:
print(len(file_items_train))
print(file_items_train[:3])
print(len(file_items_valid))

file_paths_train = [i[1] for i in file_items_train]
file_paths_valid = [i[1] for i in file_items_valid]

578
[('5_l1', '/content/drive/MyDrive/Fundus/DeepDRiD/regular-fundus-training/Images/5/5_l1.jpg', 0), ('5_l2', '/content/drive/MyDrive/Fundus/DeepDRiD/regular-fundus-training/Images/5/5_l2.jpg', 0), ('7_l2', '/content/drive/MyDrive/Fundus/DeepDRiD/regular-fundus-training/Images/7/7_l2.jpg', 0)]
58


In [ ]:
import numpy as np

In [ ]:
def make_transform(pattern):
  tsfm = transforms.Resize((512, 512))
  # add a step here to remove 10 % of the border. See if it is needed with this dataset.
  pattern= (pattern-np.min(pattern))/(np.max(pattern)-np.min(pattern))
  pattern=np.moveaxis(pattern,2,0)
 #pattern=np.expand_dims(pattern, axis=0)
  pattern_tensor=torch.from_numpy(pattern).float()
  pattern_tensor = tsfm(pattern_tensor)
  return pattern_tensor

In [ ]:
import gc

In [ ]:
# def resnet_features(model,pattern, tsfm):
#   pattern= (pattern-np.min(pattern))/(np.max(pattern)-np.min(pattern))
#   pattern=np.moveaxis(pattern,2,0)
#   pattern=np.expand_dims(pattern, axis=0)
#   pattern_tensor=torch.from_numpy(pattern).float()
#   pattern_tensor = tsfm(pattern_tensor)
#   pattern_out=model(pattern_tensor).detach().numpy()[0,:]
#   pattern_out = np.hstack(pattern_out)
#   pattern_out = np.hstack(pattern_out)

#   return pattern_out

In [ ]:
class InferDataset(torch.utils.data.Dataset):
    def __init__(self, pil_imgs):
        super(InferDataset, self,).__init__()

        self.pil_imgs = pil_imgs
        self.transform = make_transform

    def __len__(self):
        return len(self.pil_imgs)

    def __getitem__(self, idx):
        img = self.pil_imgs[idx]

        return self.transform(img)

In [ ]:
from PIL import Image

pil_imgs_train = [np.array(Image.open(path)) for path in file_paths_train]
train_data = InferDataset(pil_imgs_train)

del pil_imgs_train
gc.collect()

train_loader = torch.utils.data.DataLoader(train_data,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=4,
                                          pin_memory=True)

del train_data
gc.collect()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


0

In [ ]:
patterns = []
with torch.no_grad():
  for data in train_loader:
    data = data.to(device)
    pattern_out = model(data)
    pattern_out = pattern_out.cpu().numpy()
    patterns.append(pattern_out)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - P

In [ ]:
del train_loader
gc.collect()

patterns = np.vstack(patterns)

NameError: ignored

In [ ]:
print(patterns.shape)

(578, 1000)


In [ ]:
np.save(drid_path + '/train_patterns.npy', patterns)

In [ ]:
pil_imgs_valid = [np.array(Image.open(path)) for path in file_paths_valid]
valid_data = InferDataset(pil_imgs_valid)

del pil_imgs_valid
gc.collect()

valid_loader = torch.utils.data.DataLoader(valid_data,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=1,
                                          pin_memory=True)

del valid_data
gc.collect()

patterns = []
with torch.no_grad():
  for data in valid_loader:
    data = data.to(device)
    pattern_out = model(data)
    pattern_out = pattern_out.cpu().numpy()
    patterns.append(pattern_out)

del valid_loader
gc.collect()

patterns = np.vstack(patterns)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [ ]:
print(patterns.shape)
np.save(drid_path + '/valid_patterns.npy', patterns)

(58, 1000)


Making Database
The data to be upserted should be in following format: CIFAR100.123, [4.237038612365723, 11.179943084716797, 1.3662679195404053], {'label': 'orange'}, i.e. id, embedding, label



In [ ]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 22.4 MB/s eta 0:00:00


In [ ]:
import os
import requests

import tqdm
import httpimport
import pinecone
import numpy as np

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
os.environ["PINECONE_EXAMPLE_API_KEY"] = "5d09425a-8d43-4833-8d44-9113d452c3ed"

In [ ]:
DATA_DIRECTORY = 'tmp'
INDEX_NAME = 'image-search'
INDEX_DIMENSION = 1000
BATCH_SIZE=20

In [ ]:
pinecone.init(os.environ["PINECONE_EXAMPLE_API_KEY"], environment='us-west1-gcp-free')

# if INDEX_NAME not in pinecone.list_indexes():
#     pinecone.create_index(name=INDEX_NAME, dimension=INDEX_DIMENSION)

index = pinecone.Index(INDEX_NAME)

In [ ]:
train_dataset_path = drid_path + "/train_patterns.npy"
train_ds = np.load(train_dataset_path)
print(train_ds.shape)

(578, 1000)


In [ ]:
l_obj = []
for i in range(len(file_items_train)):
  embedding = train_ds[i]
  embedding = embedding.tolist()
  id = file_items_train[i][0]
  label = file_items_train[i][2]
  l_obj.append((id, embedding, {'label' : label}))

batches = make_batches(l_obj, BATCH_SIZE)

In [ ]:
print(len(batches))
print(len(batches[0]))
#print(batches[0])
b = batches[0][0]
print(len(b), type(b))
nda = b[1]
print(len(nda))
print(type(nda))

28
20
3 <class 'tuple'>
1000
<class 'list'>


In [ ]:
for batch in tqdm.tqdm(batches):
  index.upsert(batch)

100%|██████████| 28/28 [00:02<00:00, 10.07it/s]


In [ ]:
valid_dataset_path = drid_path + '/valid_patterns.npy'
valid_ds = np.load(valid_dataset_path)
print(valid_ds.shape)

(58, 1000)


In [ ]:
q1 = valid_ds[0].tolist()
response = index.query(q1, top_k=4, include_metadata=True)
print(response)

{'matches': [{'id': '286_r2',
              'metadata': {'label': 0.0},
              'score': 0.909316182,
              'values': []},
             {'id': '392_r1',
              'metadata': {'label': 0.0},
              'score': 0.89951539,
              'values': []},
             {'id': '7_l2',
              'metadata': {'label': 0.0},
              'score': 0.89802587,
              'values': []},
             {'id': '188_r1',
              'metadata': {'label': 0.0},
              'score': 0.891475,
              'values': []}],
 'namespace': ''}


In [ ]:
print(file_items_valid[0][2])
print(type(file_items_valid[0][2]))

0


Need to have accuracy and F1 score using confusion matrix for above. Using top-1, max-5, mindist-5, and top-5. Also need to make ROC-AUC. If good results obtained, then this is awesome, and needs to be taken to further production. Results also need to be visualized. The exact ANN used also needs to be known.

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
top_1 = []
top_5 = []
top_1_ids = []
top_5_ids = []

nbr_ids = []
nbr_labels = []
nbr_scores = []

for i in range(len(valid_ds)):
  q = valid_ds[i].tolist()
  resp = index.query(q, top_k = 5, include_metadata = True)
  nbrs_list = resp["matches"]
  top_1.append(int(nbrs_list[0]["metadata"]["label"]))
  top_1_ids.append(nbrs_list[0]["id"])

  gnd_val = file_items_valid[i][2]
  top_5_flag = True

  for nbr in nbrs_list:
    if top_5_flag and int(nbr["metadata"]["label"]) == gnd_val:
      top_5.append(int(nbr["metadata"]["label"]))
      top_5_ids.append(nbr["id"])
      top_5_flag = False
    tmp_labels.append(int(nbr["metadata"]["label"]))
    tmp_ids.append(nbr["id"])
    tmp_scores.append(nbr["score"])

  nbr_labels.append(tmp_labels)
  nbr_ids.append(tmp_ids)
  nbr_scores.append(tmp_scores)

gnd_items = [file_items_valid[i][2] for i in range(len(file_items_valid))]

In [ ]:
classification_report(gnd_items, top_1, digits = 4)

In [ ]:
classification_report(gnd_items, top_5, digits = 4)

In [ ]:
def calcGini(arr, classes):
  cd = {}
  for c in classes:
    cd[c] = 0
  for elt in arr:
    cd[elt] += 1
  tot = len(arr)
  sqsum = 0
  for c in cd:
    sqsum += (cd[c]/tot)**2
  return 2*(1 - sqsum)

In [ ]:
eta = 1e-6
classes = [0,1,2,3,4]
inv_ginis = [1/(calcGini(grp, classes) + eta) for grp in nbr_labels]

In [ ]:
acc_list = [int(top_5[i] == gnd_items[i]) for i in range(len(gnd_items))]
weighted_acc_list = [acc_list[i] * inv_ginis[i] for i in range(len(acc_list)]
print(weighted_acc_list[:10])